<a href="https://colab.research.google.com/github/Maphefo/MBTI-Personality-Classifier/blob/master/Classification_Hackathon_1074.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification Hackathon

For the specifications for today's Hackathon use the slides linked below. Just note a couple things before you start:
* Use your full name and ```_EDSA``` as your Zindi username.
* The dataset for this challenge is very large and will take a long time to process. In order to use your time wisely, only use a small subset of the data to figure out how to solve this challenge, and once you're happy with that, train your model with the entire dataset.
* This Zindi challenge is tough. This will be taken into account when the supervisors mark your work. Do not worry too much about your placement on the leaderboard. In the Regression Hackathon ```laura_the_explorer``` was in first place but is outside the top 100 in this challenge
* To submit your Hackathon to Athena, zip your notebook and your submission csv file, and upload that here. Note that your report card will say you have 100% once you submit your file.
* Please attach the *Honour code* (below) cell to your notebook. 

Further instructions found on these slides: https://docs.google.com/presentation/d/1AbVndI5aOd27Jm0E1qNoYzRtWiZ6-DE3BDE0djGxzIk/edit?usp=sharing

** Good luck! **

## Honour Code
I MMAPHEFO, SESENI, confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

In [0]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [0]:
# Load data
train = pd.read_csv('training.csv', )
test = pd.read_csv('test.csv')
df = pd.concat([train, test], axis=0, sort=True)

#store the shape for future reference
train_shape = train.shape[0]

# **Data Preprocessing**

In [399]:

print(train_shape)



95662


In [400]:
df.shape

(140681, 16)

In [401]:
train.shape

(95662, 16)

In [402]:
test.shape

(45019, 15)

In [403]:
train.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [404]:
df.head()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value
0,AccountId_3957,1000.0,BatchId_36123,ChannelId_3,256,UGX,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,SubscriptionId_887,TransactionId_76871,2018-11-15T02:18:49Z,1000
1,AccountId_4841,-20.0,BatchId_15642,ChannelId_2,256,UGX,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_73770,2018-11-15T02:19:08Z,20
2,AccountId_4229,500.0,BatchId_53941,ChannelId_3,256,UGX,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,SubscriptionId_222,TransactionId_26203,2018-11-15T02:44:21Z,500
3,AccountId_648,20000.0,BatchId_102363,ChannelId_3,256,UGX,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,SubscriptionId_2185,TransactionId_380,2018-11-15T03:32:55Z,21800
4,AccountId_4841,-644.0,BatchId_38780,ChannelId_2,256,UGX,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_28195,2018-11-15T03:34:21Z,644


In [405]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140681 entries, 0 to 45018
Data columns (total 16 columns):
AccountId               140681 non-null object
Amount                  140681 non-null float64
BatchId                 140681 non-null object
ChannelId               140681 non-null object
CountryCode             140681 non-null int64
CurrencyCode            140681 non-null object
CustomerId              140681 non-null object
FraudResult             95662 non-null float64
PricingStrategy         140681 non-null int64
ProductCategory         140681 non-null object
ProductId               140681 non-null object
ProviderId              140681 non-null object
SubscriptionId          140681 non-null object
TransactionId           140681 non-null object
TransactionStartTime    140681 non-null object
Value                   140681 non-null int64
dtypes: float64(2), int64(3), object(11)
memory usage: 18.2+ MB


In [406]:
# referenced from preprocessing : titanic-eda-wrangling-imputation 
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

        
display_all(df.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
AccountId,140681,4841,AccountId_4841,42334,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Amount,140681,NaN,NaN,NaN,6050.58,115313,-2.5e+06,-50,1000,3000,9.88e+06
BatchId,140681,139493,BatchId_67019,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ChannelId,140681,5,ChannelId_3,95025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CountryCode,140681,NaN,NaN,NaN,256,0,256,256,256,256,256
CurrencyCode,140681,1,UGX,140681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CustomerId,140681,7479,CustomerId_7343,4091,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FraudResult,95662,NaN,NaN,NaN,0.00201752,0.0448717,0,0,0,0,1
PricingStrategy,140681,NaN,NaN,NaN,2.25747,0.735089,0,2,2,2,4
ProductCategory,140681,10,financial_services,67105,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [407]:
#currency has only 1 unique value
df = df.drop(columns='CurrencyCode')
df.head() 

,AccountId,Amount,BatchId,ChannelId,CountryCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value
0,AccountId_3957,1000.0,BatchId_36123,ChannelId_3,256,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,SubscriptionId_887,TransactionId_76871,2018-11-15T02:18:49Z,1000
1,AccountId_4841,-20.0,BatchId_15642,ChannelId_2,256,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_73770,2018-11-15T02:19:08Z,20
2,AccountId_4229,500.0,BatchId_53941,ChannelId_3,256,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,SubscriptionId_222,TransactionId_26203,2018-11-15T02:44:21Z,500
3,AccountId_648,20000.0,BatchId_102363,ChannelId_3,256,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,SubscriptionId_2185,TransactionId_380,2018-11-15T03:32:55Z,21800
4,AccountId_4841,-644.0,BatchId_38780,ChannelId_2,256,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_28195,2018-11-15T03:34:21Z,644


In [408]:
# Checking how many are fraud and how many are not
df['FraudResult'].value_counts()

0.0    95469
1.0      193
Name: FraudResult, dtype: int64

In [0]:
# https://www.guru99.com/date-time-and-datetime-classes-in-python.html

df["TransactionStartTime"] = pd.to_datetime(train["TransactionStartTime"], format='%Y-%m-%d')
df[ "Transaction_Day"] =  df.TransactionStartTime.dt.day
df[ "Transaction_Month"] =  df.TransactionStartTime.dt.month
df[ "Transaction_dow"] =  df.TransactionStartTime.dt.weekday
df[ "Transaction_Week"] =  df.TransactionStartTime.dt.week
df[ "Transaction_Hour"] =  df.TransactionStartTime.dt.hour
df[ "Transaction_Minute"] =  df.TransactionStartTime.dt.minute

In [410]:
df.head()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value,Transaction_Day,Transaction_Month,Transaction_dow,Transaction_Week,Transaction_Hour,Transaction_Minute
0,AccountId_3957,1000.0,BatchId_36123,ChannelId_3,256,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,SubscriptionId_887,TransactionId_76871,2018-11-15 02:18:49+00:00,1000,15,11,3,46,2,18
1,AccountId_4841,-20.0,BatchId_15642,ChannelId_2,256,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_73770,2018-11-15 02:19:08+00:00,20,15,11,3,46,2,19
2,AccountId_4229,500.0,BatchId_53941,ChannelId_3,256,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,SubscriptionId_222,TransactionId_26203,2018-11-15 02:44:21+00:00,500,15,11,3,46,2,44
3,AccountId_648,20000.0,BatchId_102363,ChannelId_3,256,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,SubscriptionId_2185,TransactionId_380,2018-11-15 03:32:55+00:00,21800,15,11,3,46,3,32
4,AccountId_4841,-644.0,BatchId_38780,ChannelId_2,256,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_28195,2018-11-15 03:34:21+00:00,644,15,11,3,46,3,34


In [411]:
df.columns

Index(['AccountId', 'Amount', 'BatchId', 'ChannelId', 'CountryCode',
       'CustomerId', 'FraudResult', 'PricingStrategy', 'ProductCategory',
       'ProductId', 'ProviderId', 'SubscriptionId', 'TransactionId',
       'TransactionStartTime', 'Value', 'Transaction_Day', 'Transaction_Month',
       'Transaction_dow', 'Transaction_Week', 'Transaction_Hour',
       'Transaction_Minute'],
      dtype='object')

In [412]:
df.columns.astype('category')

CategoricalIndex(['AccountId', 'Amount', 'BatchId', 'ChannelId', 'CountryCode',
                  'CustomerId', 'FraudResult', 'PricingStrategy',
                  'ProductCategory', 'ProductId', 'ProviderId',
                  'SubscriptionId', 'TransactionId', 'TransactionStartTime',
                  'Value', 'Transaction_Day', 'Transaction_Month',
                  'Transaction_dow', 'Transaction_Week', 'Transaction_Hour',
                  'Transaction_Minute'],
                 categories=['AccountId', 'Amount', 'BatchId', 'ChannelId', 'CountryCode', 'CustomerId', 'FraudResult', 'PricingStrategy', ...], ordered=False, dtype='category')

In [0]:
def percent_missing(df):
    # your code here
    percent_missing = df.isnull().sum() * 100 / len(df)
    
    return dict(round(percent_missing,2))

In [414]:
percent_missing(df)

{'AccountId': 0.0,
 'Amount': 0.0,
 'BatchId': 0.0,
 'ChannelId': 0.0,
 'CountryCode': 0.0,
 'CustomerId': 0.0,
 'FraudResult': 32.0,
 'PricingStrategy': 0.0,
 'ProductCategory': 0.0,
 'ProductId': 0.0,
 'ProviderId': 0.0,
 'SubscriptionId': 0.0,
 'TransactionId': 0.0,
 'TransactionStartTime': 0.0,
 'Transaction_Day': 0.0,
 'Transaction_Hour': 0.0,
 'Transaction_Minute': 0.0,
 'Transaction_Month': 0.0,
 'Transaction_Week': 0.0,
 'Transaction_dow': 0.0,
 'Value': 0.0}

In [0]:
# storing for reference in the model since some columns will be dropped
FraudResult = train["FraudResult"]
TransactionId = test["TransactionId"]

In [0]:
# dropping unnecessary columns
# referenced from github 
df.drop(columns=["TransactionId", "AccountId","ProductId", "SubscriptionId", "BatchId", "CustomerId", "FraudResult", "CountryCode", "TransactionStartTime"], inplace = True)

In [417]:
df.head()

,Amount,ChannelId,PricingStrategy,ProductCategory,ProviderId,Value,Transaction_Day,Transaction_Month,Transaction_dow,Transaction_Week,Transaction_Hour,Transaction_Minute
0,1000.0,ChannelId_3,2,airtime,ProviderId_6,1000,15,11,3,46,2,18
1,-20.0,ChannelId_2,2,financial_services,ProviderId_4,20,15,11,3,46,2,19
2,500.0,ChannelId_3,2,airtime,ProviderId_6,500,15,11,3,46,2,44
3,20000.0,ChannelId_3,2,utility_bill,ProviderId_1,21800,15,11,3,46,3,32
4,-644.0,ChannelId_2,2,financial_services,ProviderId_4,644,15,11,3,46,3,34


In [418]:
# rearranging to categorical and numerical
df = df[['ProviderId', 'ProductCategory', 'ChannelId', 'Amount','Value','PricingStrategy','Transaction_Day','Transaction_Month','Transaction_dow','Transaction_Hour']]
df.head()

,ProviderId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,Transaction_Day,Transaction_Month,Transaction_dow,Transaction_Hour
0,ProviderId_6,airtime,ChannelId_3,1000.0,1000,2,15,11,3,2
1,ProviderId_4,financial_services,ChannelId_2,-20.0,20,2,15,11,3,2
2,ProviderId_6,airtime,ChannelId_3,500.0,500,2,15,11,3,2
3,ProviderId_1,utility_bill,ChannelId_3,20000.0,21800,2,15,11,3,3
4,ProviderId_4,financial_services,ChannelId_2,-644.0,644,2,15,11,3,3


In [0]:
# seperating between categorical and numerical
cat_columns = ["ProviderId", "ProductCategory", "ChannelId", "PricingStrategy"]
num_colums = ["Amount", "Value", "Transaction_Day","Transaction_Month", "Transaction_dow", "Transaction_Hour"]

In [0]:
# referenced from github
def standardizevar(x):
  val = (x- np.mean(x))/ np.std(x)
  return(val)
    

for col in num_colums:
  df[col] = pd.to_numeric(df[col])
df[num_colums] = df[num_colums].apply(standardizevar)

In [421]:
df.head()

,ProviderId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,Transaction_Day,Transaction_Month,Transaction_dow,Transaction_Hour
0,ProviderId_6,airtime,ChannelId_3,-0.043799,-0.078466,2,-0.156082,0.563406,-0.00578,-2.153032
1,ProviderId_4,financial_services,ChannelId_2,-0.052644,-0.086983,2,-0.156082,0.563406,-0.00578,-2.153032
2,ProviderId_6,airtime,ChannelId_3,-0.048135,-0.082812,2,-0.156082,0.563406,-0.00578,-2.153032
3,ProviderId_1,utility_bill,ChannelId_3,0.120970,0.102305,2,-0.156082,0.563406,-0.00578,-1.946419
4,ProviderId_4,financial_services,ChannelId_2,-0.058056,-0.081560,2,-0.156082,0.563406,-0.00578,-1.946419


In [0]:
# Label Encode
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
for feature in cat_columns:
    df[feature] = lab.fit_transform(df[feature])

In [423]:
df.head()

,ProviderId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,Transaction_Day,Transaction_Month,Transaction_dow,Transaction_Hour
0,5,0,2,-0.043799,-0.078466,2,-0.156082,0.563406,-0.00578,-2.153032
1,3,2,1,-0.052644,-0.086983,2,-0.156082,0.563406,-0.00578,-2.153032
2,5,0,2,-0.048135,-0.082812,2,-0.156082,0.563406,-0.00578,-2.153032
3,0,9,2,0.120970,0.102305,2,-0.156082,0.563406,-0.00578,-1.946419
4,3,2,1,-0.058056,-0.081560,2,-0.156082,0.563406,-0.00578,-1.946419


In [424]:
# https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18
from sklearn.utils import resample

X_train = df[:train_shape]
X_test = df[train_shape:]

# setting up testing and training sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, FraudResult, test_size=0.25, random_state=27)

# concatenate our training data back together
train_section = pd.concat([xtrain, ytrain], axis=1)

# separate minority and majority classes
not_fraud = train_section[train_section.FraudResult==0]
fraud = train_section[train_section.FraudResult==1];fraud.shape 

# upsample minority
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_fraud, fraud_upsampled])

# check new class counts
upsampled.FraudResult.value_counts()


1    81142
0    81142
Name: FraudResult, dtype: int64

In [0]:
ytrain = upsampled.FraudResult
xtrain = upsampled.drop('FraudResult', axis=1)

In [426]:
# https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18
from sklearn.metrics import accuracy_score, f1_score, recall_score

upsampled = LogisticRegression(solver='liblinear').fit(xtrain, ytrain)

upsampled_pred = upsampled.predict(X_test)

# Checking accuracy
print(accuracy_score(y_test, upsampled_pred))
    
# f1 score
print(f1_score(y_test, upsampled_pred))
    
print(recall_score(y_test, upsampled_pred))

0.9923064057534705
0.3566433566433566
0.9807692307692307


In [0]:
# downsample majority

#not_fraud_downsampled = resample(not_fraud,
           #                     replace = False, # sample without replacement
           #                     n_samples = len(fraud), # match minority n
           #                     random_state = 27) # reproducible results

# combine minority and downsampled majority
#downsampled = pd.concat([not_fraud_downsampled, fraud])

# checking counts
#downsampled.FraudResult.value_counts()

In [0]:
#Oversample fraud
#fraud_oversampled = resample(fraud,
#                           replace=True, 
#                           n_samples=len(not_fraud),
#                           random_state=1991) 

# #Append two datasets
#upsampled = pd.concat([not_fraud, fraud_oversampled])
#upsampled.FraudResult.value_counts()

In [0]:
#from imblearn.over_sampling import SMOTE

#X_train = df[:train_shape]
#X_test = df[train_shape:]
# setting up testing and training sets
#X_train, X_test, y_train, y_test = train_test_split(X_train, FraudResult, test_size=0.25, random_state=27)

#sm = SMOTE(random_state=27, ratio=1.0)
#X_train, y_train = sm.fit_sample(X_train, y_train)

In [379]:
#smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)

#smote_pred = smote.predict(X_test)

# Checking accuracy
#print(accuracy_score(y_test, smote_pred))

# f1 score
#print(f1_score(y_test, smote_pred))

#print(recall_score(y_test, smote_pred))

0.9918046496069577
0.3422818791946308
0.9807692307692307


In [428]:
xtrain.shape

(162284, 10)

In [429]:
FraudResult.shape

(95662,)

In [430]:
ytest.shape

(14350,)

In [0]:
model_rand = RandomForestClassifier(random_state = 1991,max_depth= 8,  n_estimators=500)

In [432]:
model_rand.fit(xtrain,ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=1991,
                       verbose=0, warm_start=False)

In [0]:
pred = model_rand.predict(X_test)

In [0]:
# delete

import sklearn.metrics as metmod
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report, confusion_matrix


def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])

def key_metrics(labels, probs, threshold):
    scores = score_model(probs, threshold)
    metrics = metmod.precision_recall_fscore_support(labels, scores)
    conf = metmod.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    {}'.format(conf[0,0]) + '             {}'.format(conf[0,1]))
    print('Actual negative     {}'.format(conf[1,0]) + '             {}'.format(conf[1,1]))
    print('')
    print('Accuracy        {}'.format(round(metmod.accuracy_score(labels, scores), 2)))
    print('MCC        {}'.format(round(metmod.matthews_corrcoef(labels, scores), 2)))
    print('AUC              {}'.format(round(metmod.roc_auc_score(labels, probs[:,1]), 2)))
    print('Macro precision {}'.format(round(float((float(metrics[0][0]) + float(metrics[0][1]))/2.0), 2)))
    print('Macro recall    {}'.format(round(float((float(metrics[1][0]) + float(metrics[1][1]))/2.0), 2)))
    
    print(' ')
    print('           Positive      Negative')
    print('Num case   {}'.format(round(metrics[3][0], 2)) + '        {}'.format(round(metrics[3][1], 2)))
    print('Precision  {}'.format(round(metrics[0][0], 2)) + '         {}'.format(round(metrics[0][1], 2)))
    print('Recall      {}'.format(round(metrics[1][0],2)) + '        {}'.format(round(metrics[1][1], 2)))
    print('F1         {}'.format(round(metrics[2][0], 2)) + '         {}' .format(round(metrics[2][1],2)))
    

In [436]:
probabilities = model_rand.predict_proba(xtest)
key_metrics(ytest, probabilities, 0.5) 

                 Confusion matrix
                 Score positive    Score negative
Actual positive    14317             10
Actual negative     0             23

Accuracy        1.0
MCC        0.83
AUC              1.0
Macro precision 0.85
Macro recall    1.0
 
           Positive      Negative
Num case   14327        23
Precision  1.0         0.7
Recall      1.0        1.0
F1         1.0         0.82


In [0]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix

In [386]:
#model = LogisticRegression()
#model.fit(xtrain, ytrain.values.ravel())

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [438]:
from google.colab import files
import io

pred_df  = pd.DataFrame({'TransactionId':TransactionId ,'FraudResult': pred})

pred_df.to_csv('sub.csv', index=False) 
files.download('sub.csv')

ValueError: ignored